In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [3]:
import grader

# SQL Miniproject

## Introduction

The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 [here](s3://dataincubator-course/coursedata/nyc_inspection_data.zip).

The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the datafiles.  Take a look and then load the csv formatted `*.txt` files into
a database as five tables:
1. `actions`
2. `cuisines`
3. `violations`
4. `grades` (from `WebExtract.txt`)
5. `boroughs` (from `RI_Webextract_BigApps_Latest.xls`)

## SQLite3

It is recommended to use sqlite3 for this project.  You can use the sqlite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [4]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

William|Shakespeare|1616
Francis|Fitzgerald|1896


Finally, we use the [ipython-sql extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loading the sql extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

In [5]:
%load_ext sql

/opt/conda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [6]:
%%sql sqlite://
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

Done.
Done.
1 rows affected.
1 rows affected.
Done.


first_name,last_name,year
William,Shakespeare,1616
Francis,Fitzgerald,1896


## Loading data


The Sqlite3 has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain malformatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some sql magic.

In [7]:
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit

In [8]:
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;

Done.
Done.


index,Name,Age
0,Alice,3
1,Bob,10


## Question 1: score_by_zipcode

Return a list of tuples of the form:

    (zipcode, mean score, number of restaurants)

for each of the 92 zipcodes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this dataset. Think about what we're actually calculating - does it represent what we're trying to understand about these zipcodes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 25,232;
- Total restaurants in valid zipcodes: 20,349

## Read the data and convert str to dataframe

In [205]:
import zipfile
import numpy as np
zf = zipfile.ZipFile('nyc_inspection_data.zip')
data = zf.read('WebExtract.txt')
# type(data)
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

import pandas as pd

TESTDATA=StringIO(data)

df = pd.read_csv(TESTDATA, sep=",")

In [142]:
# len(set(df['CAMIS']))
df.head()
# set(df['INSPDATE'])
# len(df['ZIPCODE'].dropna())

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462.0,7188924968,8,2014-03-03 00:00:00,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225.0,7182875005,39,2014-07-01 00:00:00,F,06A,23.0,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22 00:00:00,D,10B,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2014-05-29 00:00:00,D,08C,10.0,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
6,40356018,RIVIERA CATERER,3,2780,STILLWELL AVENUE,11224.0,7183723031,3,2014-06-10 00:00:00,D,10F,5.0,A,2014-06-10 00:00:00,2014-09-04 06:01:28.403000000


In [167]:
# Get the unique CAMIS
df_new = df.drop_duplicates(['CAMIS'])

# Groupby
count_table = df_new.groupby(['ZIPCODE']).count()
major_zipcode = count_table[count_table['CAMIS']>100].index

# Get average score
major_table = df_new[df['ZIPCODE'].isin(major_zipcode)]
avg_score = major_table.groupby(['ZIPCODE']).mean()
l2 = list(avg_score['SCORE'])

# Get resaurant count
major_count = major_table.groupby(['ZIPCODE']).count()
l3 = list(major_count['CAMIS'])

# Joint table
l1 = list(avg_score.index)
l1_strings = ["%.0f" % number for number in l1]
score_zipcode = zip(l1_strings,l2,l3)
# type(score_zipcode)

# Sort the order by mean score
score_zipcode = sorted(score_zipcode,key=lambda x:x[1])


/opt/conda/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [168]:
score_zipcode

[('10001', 8.7858585858585858, 509),
 ('10451', 8.9415584415584419, 157),
 ('11430', 9.4000000000000004, 148),
 ('11369', 9.6288659793814428, 104),
 ('10461', 9.724832214765101, 154),
 ('10462', 9.8137931034482762, 148),
 ('10452', 9.8586956521739122, 101),
 ('11237', 9.8914285714285715, 179),
 ('11217', 9.9715447154471537, 254),
 ('11236', 10.066037735849056, 111),
 ('11201', 10.066066066066066, 345),
 ('11231', 10.066225165562914, 159),
 ('10023', 10.067708333333334, 197),
 ('11234', 10.082758620689654, 152),
 ('10301', 10.13265306122449, 103),
 ('10468', 10.137254901960784, 107),
 ('10472', 10.140186915887851, 108),
 ('10463', 10.215999999999999, 128),
 ('11106', 10.279220779220779, 156),
 ('10036', 10.300000000000001, 593),
 ('10458', 10.303191489361701, 196),
 ('10019', 10.320987654320987, 660),
 ('11209', 10.33201581027668, 264),
 ('11215', 10.401114206128133, 372),
 ('11368', 10.417857142857143, 290),
 ('10022', 10.450317124735729, 488),
 ('10306', 10.495238095238095, 111),
 ('1

In [169]:
def score_by_zipcode():
    return score_zipcode

grader.score('sql__score_by_zipcode', score_by_zipcode)

Your score:  0.869565217391


## Question 2: score_by_map

The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zipcode](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the countryfield.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

In [92]:
x = pd.DataFrame(score_zipcode, columns=['ZIPCODE', 'Avg_Score', 'Count'])
x.to_csv('test.csv')

In [93]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://fzn0728.carto.com/builder/c1a67e0a-173c-11e7-8e35-0e3ebc282e83/embed"

grader.score('sql__score_by_map', score_by_map)

Your score:  1.0


## Question 3: score_by_borough
Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 25,220

In [157]:
# Groupby
mean_score = df[['SCORE','BORO']].groupby(['BORO']).mean()
number_restaurant = df[['CAMIS','BORO']].groupby(['BORO']).count()
borough = list(('x','MANHATTAN','THE BRONX','BROOKLYN','QUEENS','STATEN ISLAND'))
output = zip(borough,list(mean_score['SCORE']),list(number_restaurant['CAMIS']))
output[1:6]


[('MANHATTAN', 10.863070539419088, 10201),
 ('THE BRONX', 10.258347978910368, 2365),
 ('BROOKLYN', 10.925818057221175, 6072),
 ('QUEENS', 11.257068933653668, 5625),
 ('STATEN ISLAND', 11.447252747252747, 957)]

In [158]:
def score_by_borough():
    return output[1:6]

grader.score('sql__score_by_borough', score_by_borough)

Your score:  0.9


## Question 4: score_by_cuisine

Return a list of the 75 tuples of the form

    (cuisine, mean score, number of reports)

for each of the 75 cuisine types with at least 100 violation reports. Sort the list in ascending order by score. Are the least sanitary and most sanitary
cuisine types surprising?

**Note:** It's interesting to think again about what this analysis is trying to say and how it differs from the analysis by zipcode. How should this
affect the calculation in your opinion?

**Checkpoint:**
- Total entries from valid cuisines: 531,529

In [207]:
# Read Cuisine data
cuisine_data = zf.read('Cuisine.txt')
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

import pandas as pd

CUISINEDATA=StringIO(cuisine_data)

CUISINE = pd.read_csv(CUISINEDATA, sep=",")
CUISINE.head()

,CUISINECODE,CODEDESC
0,2,African
1,3,American
2,5,Asian
3,15,Cajun
4,17,Caribbean


In [210]:
# mean_score = df[['SCORE','CUISINECODE']].groupby(['CUISINECODE']).mean()
# df = df.drop_duplicates(['CAMIS'])
count_table_cuisine = df.groupby(['CUISINECODE']).count()
cuisine_viol = count_table_cuisine[count_table_cuisine['VIOLCODE']>=100].index
cuisine_table = df[df['CUISINECODE'].isin(cuisine_viol)]


number_restaurant_cuisine = cuisine_table[['VIOLCODE','CUISINECODE']].groupby(['CUISINECODE']).count()
mean_score_cuisine = cuisine_table[['SCORE','CUISINECODE']].groupby(['CUISINECODE']).mean()

cuisine_name = CUISINE[CUISINE['CUISINECODE'].isin(mean_score_cuisine.index)]['CODEDESC']

output_cuisine = zip(list(cuisine_name),list(mean_score_cuisine['SCORE']),list(number_restaurant_cuisine['VIOLCODE']))

# Sort
output_cuisine = sorted(output_cuisine,key=lambda x:x[1])

output_cuisine
# output[1:6]
# mean_score_cuisine

# rpt[rpt['STK_ID'].isin(stk_list)]
# Get average score
# major_table = df_new[df['ZIPCODE'].isin(major_zipcode)]_

[('Chinese/Cuban', 14.51923076923077, 145),
 ('Seafood', 14.961199294532628, 576),
 ('Bakery', 15.346153846153847, 297),
 ('Afghan', 15.536336731338375, 6303),
 ('Chicken', 15.5413870246085, 433),
 ('Chinese/Japanese', 15.943380040034315, 3670),
 ('Portuguese', 16.438442211055275, 6510),
 ('English', 16.8989898989899, 2638),
 ('Italian', 17.125518307681496, 14860),
 ('Salads', 17.19119623655914, 3057),
 ('Japanese', 17.214285714285715, 179),
 ('Caribbean', 17.233449477351915, 632),
 ('Not Listed/Not Applicable', 17.392756083757781, 1881),
 ('Caf\xe9/Coffee/Tea', 17.512352309344791, 7663),
 ('Indian', 17.851214574898787, 1021),
 ('Australian', 18.541935483870969, 348),
 ('Bagels/Pretzels', 18.644654088050313, 334),
 ('Soul Food', 19.211340206185568, 203),
 ('Korean', 19.317600227466592, 7442),
 ('Sandwiches', 20.083333333333332, 149),
 ('Polish', 20.160220994475139, 552),
 ('Hotdogs/Pretzels', 20.265353805073431, 3313),
 ('Bottled beverages, including water, sodas, juices, etc.',
  20.5

In [209]:
def score_by_cuisine():
    return output_cuisine

grader.score('sql__score_by_cuisine', score_by_cuisine)

Your score:  0.0666666666667


## Question 5: violation_by_cuisine
Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy becuase you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just becuase they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restuarants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stackoverflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in Violation.txt).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.37009216349859

In [ ]:
def violation_by_cuisine():
    return [(("Café/Coffee/Tea",
              "Toilet facility not maintained and provided with toilet paper; "
              "waste receptacle and self-closing door."),
             1.87684775827172, 315)] * 20

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*